link: https://www.kaggle.com/kswamy15/mercari-using-pytorch

In [1]:
import torch
from torch.autograd import Variable
from torch import optim
from torch.optim import lr_scheduler
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F

In [2]:
import numpy as np
import pandas as pd
import time

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import math

In [4]:
train_df = pd.read_csv('data/mercari/train.tsv', sep='\t')
test_df = pd.read_csv('data/mercari/test.tsv', sep='\t')

In [5]:
train_df.describe()

,train_id,item_condition_id,price,shipping
count,1.482535e+06,1.482535e+06,1.482535e+06,1.482535e+06
mean,7.412670e+05,1.907380e+00,2.673752e+01,4.472744e-01
std,4.279711e+05,9.031586e-01,3.858607e+01,4.972124e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,3.706335e+05,1.000000e+00,1.000000e+01,0.000000e+00
50%,7.412670e+05,2.000000e+00,1.700000e+01,0.000000e+00
75%,1.111900e+06,3.000000e+00,2.900000e+01,1.000000e+00
max,1.482534e+06,5.000000e+00,2.009000e+03,1.000000e+00


In [6]:
train_df.shape

(1482535, 8)

In [7]:
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [8]:
print('Train columns with null values:\n', train_df.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', test_df.isnull().sum())
print("-"*10)

train_df.describe(include = 'all')

Train columns with null values:
 train_id                  0
name                      0
item_condition_id         0
category_name          6327
brand_name           632682
price                     0
shipping                  0
item_description          4
dtype: int64
----------
Test/Validation columns with null values:
 test_id                   0
name                      0
item_condition_id         0
category_name          3058
brand_name           295525
shipping                  0
item_description          0
dtype: int64
----------


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
count,1.482535e+06,1482535,1.482535e+06,1476208,849853,1.482535e+06,1.482535e+06,1482531
unique,NaN,1225273,NaN,1287,4809,NaN,NaN,1281426
top,NaN,Bundle,NaN,"Women/Athletic Apparel/Pants, Tights, Leggings",PINK,NaN,NaN,No description yet
freq,NaN,2232,NaN,60177,54088,NaN,NaN,82489
mean,7.412670e+05,NaN,1.907380e+00,NaN,NaN,2.673752e+01,4.472744e-01,NaN
std,4.279711e+05,NaN,9.031586e-01,NaN,NaN,3.858607e+01,4.972124e-01,NaN
min,0.000000e+00,NaN,1.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,NaN
25%,3.706335e+05,NaN,1.000000e+00,NaN,NaN,1.000000e+01,0.000000e+00,NaN
50%,7.412670e+05,NaN,2.000000e+00,NaN,NaN,1.700000e+01,0.000000e+00,NaN
75%,1.111900e+06,NaN,3.000000e+00,NaN,NaN,2.900000e+01,1.000000e+00,NaN


Calculate how much of the brand names are not there.

In [9]:
print(632682/train_df.shape[0])

0.4267568725190299


Around 50% is not there hence we should probably not consider this.

In [10]:
train_df = train_df.drop(['brand_name'], axis=1)
test_df = test_df.drop(['brand_name'], axis=1)

# categories

We need to check how many categories there are

In [11]:
# Merge the two dataframes
frames = [train_df, test_df]
combined_df = pd.concat(frames)

In [12]:
combined_cat_df = combined_df['category_name']
def split_cat(text):
    try: return text.split("/")
    except: pass

combined_cat_df = combined_cat_df.apply(lambda x: split_cat(x))

In [13]:
def no_of_cats(cat_list):
    try: return len(cat_list)
    except: return 0
    
no_of_cats = pd.DataFrame(combined_cat_df.apply(lambda x: no_of_cats(x)))

In [14]:
# no_of_cats['category_name'].max(axis=1)
index_whr_max_categories = no_of_cats['category_name'].argmax()
print(index_whr_max_categories)
max_num_of_categories = len(split_cat(combined_df.iloc[[index_whr_max_categories]]['category_name'].tolist()[0]))
print('there are a maximum of {} categories and this is happened in row:'.format(max_num_of_categories))
combined_df.iloc[[index_whr_max_categories]]

239
there are a maximum of 5 categories and this is happened in row:


,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id
239,Electronics/Computers & Tablets/iPad/Tablet/eB...,1,Zag invisible shield for IPad air,Zagg invisible shield for IPad air,10.0,1,NaN,239.0


In [15]:
def split_cat(text, max_num_of_categories):
    return_val = ["None"] * max_num_of_categories
    try:
        text_list = text.split("/") + return_val
        return text_list[:max_num_of_categories]
    except:
        return return_val

Change the category name for train and test and total dataframes

In [16]:
train_df['category_name'] = train_df['category_name'].apply(lambda x: split_cat(x, max_num_of_categories))
test_df['category_name'] = test_df['category_name'].apply(lambda x: split_cat(x, max_num_of_categories))
combined_df['category_name'] = combined_df['category_name'].apply(lambda x: split_cat(x, max_num_of_categories))

In [17]:
train_df.head()

,train_id,name,item_condition_id,category_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,"[Men, Tops, T-shirts, None, None]",10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,"[Electronics, Computers & Tablets, Components ...",52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,"[Women, Tops & Blouses, Blouse, None, None]",10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,"[Home, Home Décor, Home Décor Accents, None, N...",35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,"[Women, Jewelry, Necklaces, None, None]",44.0,0,Complete with certificate of authenticity


now we know that there are 5 categories so we will try to find the unknown ones category per category. so we will make predictions based on the 5 categories

### Running category encoding on the first category

In [31]:
combined_cat1_list = [x[0] for x in combined_df['category_name'].tolist()]
combined_cat1_list[:5]

['Men', 'Electronics', 'Women', 'Home', 'Women']

In [32]:
cat1_le = LabelEncoder()
cat1_le.fit(combined_cat1_list)

LabelEncoder()

In [33]:
cat1_le.transform(['Men', 'Electronics', 'Women', 'Home', 'Women'])

array([ 5,  1, 10,  3, 10])

In [34]:
cat1_le.inverse_transform([ 5,  1, 10,  3, 10])

array(['Men', 'Electronics', 'Women', 'Home', 'Women'], 
      dtype='<U22')

Thus we are able to build a label encoder state space for the first category

In [19]:
train_df[train_df.isnull().any(axis=1)]

,train_id,name,item_condition_id,category_name,price,shipping,item_description
511535,511535,Shoes for Michelle,4,"[Kids, Girls 0-24 Mos, Shoes, None, None]",9.0,0,NaN
861230,861230,Lipgloss,4,"[Beauty, Makeup, Lips, None, None]",49.0,0,NaN
1224924,1224924,Disney Minnie Head band,3,"[Women, Women's Accessories, Hair Accessories,...",9.0,0,NaN
1264242,1264242,For Bianca,3,"[Women, Women's Accessories, Scarves & Wraps, ...",10.0,1,NaN


In [20]:
train_df[train_df.isnull().any(axis=1)]

,train_id,name,item_condition_id,category_name,price,shipping,item_description
511535,511535,Shoes for Michelle,4,"[Kids, Girls 0-24 Mos, Shoes, None, None]",9.0,0,NaN
861230,861230,Lipgloss,4,"[Beauty, Makeup, Lips, None, None]",49.0,0,NaN
1224924,1224924,Disney Minnie Head band,3,"[Women, Women's Accessories, Hair Accessories,...",9.0,0,NaN
1264242,1264242,For Bianca,3,"[Women, Women's Accessories, Scarves & Wraps, ...",10.0,1,NaN


In [21]:
value_list = ['iPhone']
train_df[train_df.name.isin(value_list)]

,train_id,name,item_condition_id,category_name,price,shipping,item_description
161166,161166,iPhone,1,"[Electronics, Cell Phones & Accessories, Cell ...",4.0,1,Luxury 360° Hybrid Acrylic Hard Case Original ...
331692,331692,iPhone,2,"[Electronics, Cell Phones & Accessories, Cell ...",76.0,0,iCloud lock!!! Screen protector no scratches! ...
502203,502203,iPhone,3,"[Electronics, Cell Phones & Accessories, Cell ...",26.0,1,No description yet
901734,901734,iPhone,1,"[Electronics, Cell Phones & Accessories, Cell ...",17.0,1,Apple iPhone headphones for iPhones compatible...
968525,968525,iPhone,2,"[Electronics, Cell Phones & Accessories, Cell ...",19.0,0,Two iPhone 4s they are locked so they are to b...
1198530,1198530,iPhone,2,"[Electronics, Cell Phones & Accessories, Cell ...",116.0,0,iPhone 5c no cracks brand new unlocked (no SIM...
1220558,1220558,iPhone,4,"[Electronics, Cell Phones & Accessories, Cell ...",16.0,1,"iPhone 4 Cracked front Back up for sale, had i..."
1385743,1385743,iPhone,4,"[Electronics, Cell Phones & Accessories, Cell ...",20.0,1,It works won't turn on need a battery and new ...
1473558,1473558,iPhone,3,"[Electronics, Cell Phones & Accessories, Cell ...",80.0,1,iPhone 6 model number A1586. Found never could...


For the missing category names we should try to find some unsupervised learning so that some amount filling of the data should be present.

# Running NLP on the categories

We will first try to classify the documents and see if we can get some meaningful classification based on that.

Idea is to use only the name to predict the category name

So we will drop all the remaining columns

In [22]:
print(train_df.columns.tolist())

['train_id', 'name', 'item_condition_id', 'category_name', 'price', 'shipping', 'item_description']


In [23]:
from copy import deepcopy
category_df = deepcopy(train_df)

In [24]:
category_df = category_df.drop(['train_id', 'item_condition_id', 'price', 'shipping'], axis=1)

In [25]:
category_df.sample(2)

,name,category_name,item_description
356360,Southern prep shirt *on hold for ms*,"[Women, Tops & Blouses, T-Shirts, None, None]",Very soft and only worn once
626803,bundle,"[Beauty, Makeup, Makeup Sets, None, None]",brand new NO FREE SHIP


In [26]:
predict_category_df = category_df[pd.isnull(category_df['category_name'])]
train_test_categry_df = category_df[pd.notnull(category_df['category_name'])]
train_categry_df, test_categry_df = train_test_split(train_test_categry_df, test_size=0.2, random_state=42)
print('separated into predict, train and test')
print(category_df.shape, predict_category_df.shape, train_categry_df.shape, test_categry_df.shape)
print(predict_category_df.shape[0] + train_categry_df.shape[0] + test_categry_df.shape[0])

separated into predict, train and test
(1482535, 3) (0, 3) (1186028, 3) (296507, 3)
1482535


In [27]:
X_train_category_df = train_categry_df[['name', 'item_description']]
y_train_category_df = train_categry_df[['category_name']]
X_test_category_df = test_categry_df[['name', 'item_description']]
y_test_category_df = test_categry_df[['category_name']]
print('separate to x and y')
print(X_train_category_df.shape, y_train_category_df.shape, X_test_category_df.shape, y_test_category_df.shape)

separate to x and y
(1186028, 2) (1186028, 1) (296507, 2) (296507, 1)


category names are based on parent -> sub category -> subcategory etc. Need to find how many categories are there.

In [28]:
y_train_category_df.head()

,category_name
1416089,"[Kids, Boys (4+), Bottoms, None, None]"
1423955,"[Other, Other, Other, None, None]"
403867,"[Beauty, Hair Care, Styling Products, None, None]"
701974,"[Women, Pants, Casual Pants, None, None]"
1124330,"[Women, Underwear, Panties, None, None]"


In [29]:
X_category_df = category_df[['name', 'item_description']]
y_category_df = category_df[['category_name']]